In [1]:
#pip install git+https://github.com/technige/py2neo.git@master#egg=py2neo

  Cloning https://github.com/technige/py2neo.git (to revision master) to /tmp/pip-install-c8c7dc0f/py2neo_9b02185400e84a2eac53ffb490cb1c50
  Running command git clone -q https://github.com/technige/py2neo.git /tmp/pip-install-c8c7dc0f/py2neo_9b02185400e84a2eac53ffb490cb1c50
     |████████████████████████████████| 146 kB 937 kB/s eta 0:00:01
     |████████████████████████████████| 68 kB 4.3 MB/s eta 0:00:01
  Created wheel for py2neo: filename=py2neo-2021.1.dev0-py2.py3-none-any.whl size=204218 sha256=7162fe5672af4f7cb5d6b81dd8e6476527fde6fb8d98917a06dbf6f0d9a028f0
  Stored in directory: /tmp/pip-ephem-wheel-cache-fk12x1jg/wheels/e5/91/5d/7a3907927ffaa5a0633d874caf103b780bd731a56856cd416f
  Created wheel for neotime: filename=neotime-1.7.4-py3-none-any.whl size=20541 sha256=5374ce43f9808449c6303b98c89dba894708749c959c00e9bf26534d4976f598
  Stored in directory: /home/usrsiac/.cache/pip/wheels/84/2d/01/a397b73c9e48f6c44342c06c436b20b44d53a71f6ba972a992
Successfully built py2neo neotime
No

Leer el CSV

In [90]:
import pandas as pd
df = pd.read_csv('ResultadoCategoriaFinal.csv')
print(len(df['link']))

550


In [91]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", auth=("neo4j", "123456789"))

import re

diccionario_problemas=[]
graph.run('CALL n10s.graphconfig.init()');
graph.run('CALL n10s.rdf.import.fetch("https://gitlab.com/john_tenesaca/base-tesis/-/raw/master/vivo.owl","Turtle")');
        
for carga in range(len(df['authors'])):
    try:
        print(carga)
        

        graph.run(' merge (mi_instancia:instancia {titulo: "Instancia '+str(carga)+'"})'
        +' merge (Titulo:title {title:"'+re.sub('"',"", str(df['title'][carga]))+'"})'
        +' merge (Abstract:abstract {abstract:"'+re.sub('"',"", str(df['abstract'][carga]))   +'"})'
        +' merge (Link:link {link:"'+re.sub('"',"", str(df['link'][carga]))+'"})'
        +' merge (Fuente:fuente {fuente:"'+str(df['fuente'][carga])+'"})'  
        +' merge (Key:key {key:"'+str(df['key'][carga])+'"})'
        +' merge (mi_instancia)-[:rdfs__subClassOf]->(Titulo)'
        +' merge (mi_instancia)-[:rdfs__subClassOf]->(Abstract)'
        +' merge (mi_instancia)-[:rdfs__subClassOf]->(Link)'
        +' merge (mi_instancia)-[:rdfs__subClassOf]->(Fuente)'
        +' merge (mi_instancia)-[:rdfs__subClassOf]->(Key)'

        +' merge(valor1:owl__Class{rdfs__label:"Article"})'
        +' merge (valor1)-[:rdfs__subClassOf]->(mi_instancia)'

        #Relacion de 2 o mas autores       
        +' merge (mi_autor:autor {nombres: "Autores '+str(carga)+'"})'
        +' merge (mi_instancia)-[:rdfs__subClassOf]->(mi_autor)'  

        +' merge (mi_categoria:categoriaTitulo {caracteristicas: "Categoria '+str(carga)+'"})'
        +' merge (mi_categoriabstract:categoriaAbstract {caracteristicas: "Categoria '+str(carga)+'"})'
        
        +' merge (mi_instancia)-[:rdfs__subClassOf]->(mi_categoria)' 
        +' merge (mi_instancia)-[:rdfs__subClassOf]->(mi_categoriabstract)' 

        +' merge (mi_keyword:keyword {palabras: "Keyword '+str(carga)+'"})'
        +' merge (mi_instancia)-[:rdfs__subClassOf]->(mi_keyword)' 

                 )  
    except:
        diccionario_problemas.append(carga)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [97]:
for carga_general in range(len(df['keywords'])):  
    try: 
            #print(carga_general)
        general=df['keywords'][carga_general]
            #print("---------------")
            #print(df['keywords'][carga_general])
        output=eval(general)
            #print(carga_general)
        for j in range(len(output)):
            nombre=output[j].split(',')
                #print(nombre[0])
            graph.run(' merge (valor1:keyword{palabras:"Keyword '+str(carga_general)+'"})'
                    +' merge (valor2:palabra{nombre:"'+str(nombre[0])+'"})'
                    +' merge (valor1)-[:rdfs__subClassOf]->(valor2)')
    except:
        print("No relacionar K")


No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relacionar K
No relac

In [92]:
     
def agregar_autores(df,name):  
    for carga_general in range(len(df['authors'])):  
        try:  
            #print(carga_general)
            general=df['authors'][carga_general]
            output=eval(general)
            #print(carga_general)
            for j in range(len(output)):
                nombre=output[j].split(',')
                #print(nombre[0])
                graph.run(' merge (valor1:autor{nombres:"Autores '+str(carga_general)+'"})'
                    +' merge (valor2:persona{nombre:"'+str(nombre[0])+'"})'
                    +' merge (valor1)-[:rdfs__subClassOf]->(valor2)')
        except:
            print("No relacionar")


In [93]:
def agregar_keywords(df): 
    for carga_general in range(len(df['keywords'])):  
        try: 
            #print(carga_general)
            general=df['keywords'][carga_general]
            #print("---------------")
            #print(df['keywords'][carga_general])
            output=eval(general)
            #print(carga_general)
            for j in range(len(output)):
                nombre=output[j].split(',')
                #print(nombre[0])
                graph.run(' merge (valor1:keyword{palabras:"Keyword '+str(carga_general)+'"})'
                    +' merge (valor2:palabra{nombre:"'+str(nombre[0])+'"})'
                    +' merge (valor1)-[:rdfs__subClassOf]->(valor2)')
        except:
            print("No relacionar K")
 

In [94]:
def agregar_titulo (df,name):
    for carga_general in range(len(df['categoriatitulo'])):  
        try:  
            general=df['categoriatitulo'][carga_general]
            output=eval(general)
            #print(carga_general)
            for j in range(len(output)):
                nombre=output[j].split(',')
                #print(nombre[0])
                graph.run(' merge (valor1:categoriaTitulo{caracteristicas:"Categoria '+str(carga_general)+'"})'
                    +' merge (valor2:palabraclave{nombre:"'+str(nombre[0])+'"})'
                    +' merge (valor1)-[:rdfs__subClassOf]->(valor2)')
        except:
            print("No relacionar")

In [95]:
def  agregar_abstract(df,name):
    for carga_general in range(len(df['categoriabstract'])):  
        try:  
            general=df['categoriabstract'][carga_general]
            output=eval(general)
            #print(carga_general)
            for j in range(len(output)):
                nombre=output[j].split(',')
                #print(nombre[0])
                graph.run(' merge (valor1:categoriaAbstract{caracteristicas:"Categoria '+str(carga_general)+'"})'
                    +' merge (valor2:palabraclave{nombre:"'+str(nombre[0])+'"})'
                    +' merge (valor1)-[:rdfs__subClassOf]->(valor2)')
        except:
            print("No relacionar")


In [96]:
import threading, time
#CREAMOS PROCESOS A EJECUTAR EN PARALELO.        
t = threading.Thread(target = agregar_titulo, args =(df, 'thread1') )
t2 = threading.Thread(target = agregar_abstract, args =(df, 'thread2') )
t3 = threading.Thread(target = agregar_keywords, args =(df, 'thread3') )
t4 = threading.Thread(target = agregar_autores, args =(df, 'thread4') )


#INICIAMOS PROCESOS.
t.start()
t2.start()
t3.start()
t4.start()


Exception in thread Thread-27:
Traceback (most recent call last):
  File "/home/usrsiac/anaconda3/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/usrsiac/anaconda3/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
TypeError: agregar_keywords() takes 1 positional argument but 2 were given


No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar
No relacionar


In [9]:
#cambio a diccionario 
graph.run('match(n:owl__Class{rdfs__label:"acute conjunctivitis"})-[r:rdfs__subClassOf]->(p:owl__Class) return n, r, p UNION match(n:owl__Class{rdfs__label:"acute conjunctivitis"})-[r:rdfs__subClassOf]-(p:owl__Class) return n, r, p').data()

[{'n': Node('Resource', 'owl__Class', ns0__hasDbXref='UMLS:C0155141', ns0__hasExactSynonym='conjunctivitis (disease), acute', ns0__id='MONDO:0001214', ns2__IAO_0000115='Acute inflammation of the conjunctiva.', rdfs__label='acute conjunctivitis', uri='http://purl.obolibrary.org/obo/MONDO_0001214'),
  'r': rdfs__subClassOf(Node('Resource', 'owl__Class', ns0__hasDbXref='UMLS:C0155141', ns0__hasExactSynonym='conjunctivitis (disease), acute', ns0__id='MONDO:0001214', ns2__IAO_0000115='Acute inflammation of the conjunctiva.', rdfs__label='acute conjunctivitis', uri='http://purl.obolibrary.org/obo/MONDO_0001214'), Node('Resource', 'owl__Class', ns0__hasDbXref='UMLS:C0001314', ns0__hasExactSynonym='acute disease', ns0__hasRelatedSynonym='acute disease', ns0__id='MONDO:0020683', ns2__IAO_0000115='Disease having a short and relatively severe course.', rdfs__label='acute disease', uri='http://purl.obolibrary.org/obo/MONDO_0020683')),
  'p': Node('Resource', 'owl__Class', ns0__hasDbXref='UMLS:C000

In [10]:
#creacion en dataFrame
graph.run('match(n:owl__Class{rdfs__label:"acute conjunctivitis"})-[r:rdfs__subClassOf]->(p:owl__Class) return n, r, p UNION match(n:owl__Class{rdfs__label:"acute conjunctivitis"})-[r:rdfs__subClassOf]-(p:owl__Class) return n, r, p').to_data_frame()

,n,r,p
0,{'ns0__hasExactSynonym': 'conjunctivitis (dise...,{},"{'ns0__hasExactSynonym': 'acute disease', 'ns0..."
1,{'ns0__hasExactSynonym': 'conjunctivitis (dise...,{},"{'ns0__hasExactSynonym': 'pink eye', 'ns0__has..."
2,{'ns0__hasExactSynonym': 'conjunctivitis (dise...,{},{'ns0__hasExactSynonym': 'epidemic hemorrhagic...
3,{'ns0__hasExactSynonym': 'conjunctivitis (dise...,{},{'ns0__hasExactSynonym': 'serous conjunctiviti...
4,{'ns0__hasExactSynonym': 'conjunctivitis (dise...,{},{'ns0__hasExactSynonym': 'acute follicular con...
5,{'ns0__hasExactSynonym': 'conjunctivitis (dise...,{},{'ns0__hasExactSynonym': 'conjunctivitis with ...
6,{'ns0__hasExactSynonym': 'conjunctivitis (dise...,{},{'ns0__hasExactSynonym': 'acute atopic conjunc...
7,{'ns0__hasExactSynonym': 'conjunctivitis (dise...,{},"{'ns0__hasExactSynonym': 'pinkeye', 'ns0__hasD..."


In [17]:
from py2neo  import Node, Relationship

tx = graph.begin()
a = Node("Person", name="Alice", age=33)
b = Node("Person", name="Bob", age=44)
KNOWS = Relationship.type("KNOWS")
graph.merge(KNOWS(a, b), "Person", "name")
graph.run("MATCH (a:Person) RETURN a.name, a.born LIMIT 4").data()

[{'a.name': 'Alice', 'a.born': None}, {'a.name': 'Bob', 'a.born': None}]

In [ ]:
def creacion_neo4j(numero):
    g = Graph()
    g.run("CREATE (a:Person) SET a.name = 'Alice'").stats()

In [ ]:
def creacion_neo4j(numero):
    g = Graph()
    g.run("CREATE (a:Person) SET a.name = 'Alice'").stats()

In [ ]:


string=df['authors'][0]
output=eval(string)
output

string=output[0]
output=string.split(',')
output[0]
